In [ ]:
import numpy
from numpy import *
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive")

In [ ]:
data_raw = genfromtxt("/content/MyDrive/MyDrive/Data/PRNN/Assignment_1/p4_train.csv",delimiter = ",")

In [ ]:
X_raw,Y_raw = data_raw[1:,1:],data_raw[1:,0]
noise = 0.01*random.randn(X_raw.shape[0],X_raw.shape[1])
X_raw = X_raw/255 + noise

In [ ]:
X_raw.shape

In [ ]:
len(X_raw)

In [ ]:
X_raw1 = []

for i in range(len(X_raw)):
  X_raw1.append(X_raw[i].reshape(28,28))

In [ ]:
X_raw1[0].shape

In [ ]:
X_raw_resize = []
for i in range(len(X_raw)):
  X_raw_resize.append(resize(X_raw1[i],(10,10)))

In [ ]:
plt.imshow(X_raw1[0],cmap = "gray")
plt.show()

In [ ]:
plt.imshow(X_raw_resize[0],cmap = "gray")
plt.show()

In [ ]:
N,d = X_raw.shape
Ns = 10000

In [ ]:
random_samples = random.choice(arange(N),Ns)

In [ ]:
X_use,Y_use = X_raw[random_samples],Y_raw[random_samples]

In [ ]:
tts_ratio = 0.6
train_split = round(tts_ratio * Ns)
test_split = int(Ns-train_split)

train_split,test_split

In [ ]:
X_train,Y_train = X_use[0:train_split],Y_use[0:train_split]
X_test,Y_test = X_use[Ns-test_split:Ns],Y_use[Ns-test_split:Ns]
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

In [ ]:
X_pca = cov(X_train.transpose())

In [ ]:
eigen_values,eigen_vectors = linalg.eig(X_pca)

In [ ]:
eigenvalues,eigenvectors = linalg.eig(X_pca)

In [ ]:
eigen_indices = argsort(eigen_values)[::-1]

In [ ]:
eigenvectors_sorted = eigen_vectors[eigen_indices]

In [ ]:
eigenvectors_sorted
variance_explained = []
for i in eigen_values :
  variance_explained.append((i/sum(eigen_values))*100)
print(variance_explained)

In [ ]:
with plt.style.context("ggplot"):
  plt.figure(figsize = (6,4))
plt.bar(range(d),variance_explained,alpha = 0.5, align = "center", label = "indiviual explained variance")
plt.ylabel("Explained Variance Ratio")
plt.xlabel("Principal Components")
plt.legend()
plt.tight_layout()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 10)

X_train_pca = pca.fit_transform(X_train)


In [ ]:
X_train_pca[0].shape

In [ ]:
plt.imshow(X_train_pca[0].reshape(2,5))
plt.show()

In [ ]:
X_test_pca = pca.transform(X_test)

In [ ]:
X_test_pca.shape

In [ ]:
X_train_pca[0]-X_train[0,0:10]

In [ ]:
J = len(unique(Y_train))

x_class=[]

for j in range(J):
  idx = array(where(Y_train == j))
  x_class.append(X_train_pca[idx])
print(x_class[0][0].shape)

In [ ]:
def Gaussian_Density_fn(x,mu,sigma):

    constant = 1/sqrt(2*pi*sigma**2)
    term = exp(-(((x-mu)**2)/(2*sigma**2)))
    pdf = constant*term
    return pdf


In [ ]:
def MultivariateGaussianPDF(x,mu,sigma):

  #print(len(x))
  constant = 1/(sqrt((2*pi)**len(x))*linalg.det(sigma)**(1/2))
  term = exp(-((x-mu).reshape(len(x),1).transpose() @ linalg.inv(sigma) @ (x-mu).reshape(len(x),1))/2)

  return constant * term

In [ ]:
num_components = 3
epsilon = 10**-1
number_of_iterations = 30

In [ ]:
def EM(x):

  llh =[]
  alphas = ones((num_components,1))/num_components

  mus = random.randn(num_components,shape(x)[1])

  variances = array([eye(shape(x)[1]) for m in range(num_components)])

  pxi0 = zeros((num_components,shape(x)[0]))
  gammas = zeros((num_components,shape(x)[0]))

  for ii in range(number_of_iterations):
    if ii==0:
      for m in range(num_components):
        for i in range(shape(x)[0]):
           pxi0[m,i] = MultivariateGaussianPDF(x[i].reshape(shape(x)[1],),mus[m],variances[m])*alphas[m]
    else:
      pxi0 = pxi1

    for m in range(num_components):
      for i in range(shape(x)[0]):
        gammas[m,i] = pxi0[m,i]/numpy.sum(pxi0[:,i])

    llh1 = 0
    for i in range(shape(x)[0]):
      llh1 = llh1 + log(numpy.sum(pxi0[:,i]))
    if ii==0:
      llh.append(llh1)

    #Updation Occurs Here!!!

    for m in range(num_components):
      alphas[m] = numpy.sum(gammas[m,:])/shape(x)[0]
    #print(alphas)

    for m in range(num_components):

      sum_mu = 0
      for i in range(shape(x)[0]):
        sum_mu = sum_mu + gammas[m,i]*x[i]
      mus[m] = sum_mu/numpy.sum(gammas[m,:])
    #print(mus)

    for m in range(num_components):
      sum_var = 0
      for i in range(shape(x)[0]):
           sum_var = sum_var + gammas[m,i]*(dot((x[i]-mus[m]).reshape(shape(x)[1],1),(x[i]-mus[m]).reshape(shape(x)[1],1).transpose()))
      variances[m] = sum_var / numpy.sum(gammas[m,:])


    #print(variances)
    pxi1 = zeros((num_components,shape(x)[0]))

    for m in range(num_components):
      for i in range(shape(x)[0]):
        pxi1[m,i] = MultivariateGaussianPDF(x[i].reshape(shape(x)[1],),mus[m],variances[m])*alphas[m]    #print(pxi0)

    llh2 = 0

    for i in range(shape(x)[0]):
      llh2 = llh2 + log(numpy.sum(pxi1[:,i]))
    llh.append(llh2)

    if abs(llh[ii]-llh[ii-1])<=epsilon:
      print("At iteration : {0}".format(ii),"our EM Algorithm Converges")
      print("And the loglikelyhood (till this iteration {0}) is :".format(ii+1),llh)
      break
    elif(ii==number_of_iterations-1):
      print("the loglikelyhood for the given class is (for all iterations till {0}) :".format(ii+1),llh)

  return alphas,mus,variances


In [ ]:
def train(data,classes):

  J = len(unique(classes))
  x_class=[]

  for j in range(J):
    idx = array(where(classes == j))
    x_class.append(data[idx])

  ALPHA = []
  MU = []
  SIGMA = []
  for j in range(J):
    print("Class {0}".format(j))
    alpha,mu,sigma = EM(x_class[j][0])
    ALPHA.append(alpha)
    MU.append(mu)
    SIGMA.append(sigma)
  return ALPHA , MU, SIGMA


In [ ]:
ALPHA,MU,SIGMA = train(X_train_pca,Y_train)

In [ ]:
ALPHA[0].shape

In [ ]:
def test(xtest):

  pxi = zeros((J,shape(xtest)[0]))

  for j in range(J):
    for i in range(shape(xtest)[0]):
      temparr = zeros((num_components,1))
      for m in range(num_components):
        temparr[m] = MultivariateGaussianPDF(xtest[i],MU[j][m],SIGMA[j][m])
      pxi[j,i] = dot(ALPHA[j].transpose() , temparr)

  y_pred = zeros((shape(xtest)[0],1))

  for i in range(shape(xtest)[0]):
    y_pred[i] = argmax(pxi[:,i])

  return y_pred

In [ ]:
y_pred = test(X_test_pca)

In [ ]:
y_pred

In [ ]:
def evaluation(ytest):
    c=0

    for i in range(shape(ytest)[0]):
      if y_pred[i] == ytest[i]:
        c = c+1

    accuracy = (c/shape(ytest)[0])*100

    print("Accuracy:",accuracy)

    return accuracy

In [ ]:
evaluate = evaluation(Y_test)